In [1]:
import pandas as pd
from pandas import DataFrame 

In [2]:
df_tennis = pd.read_csv('exam.csv')
print(df_tennis)

   A  B  Y
0  0  0  0
1  0  1  0
2  1  0  1
3  1  1  0


In [3]:
def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )


In [4]:
def entropy_of_list(a_list):  
    from collections import Counter
    cnt = Counter(x for x in a_list)   # Counter calculates the propotion of class
    num_instances = len(a_list)*1.0   # = 14
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO
    return entropy(probs) # Call Entropy :
total_entropy = entropy_of_list(df_tennis['Y'])


In [5]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index) * 1.0
    print("split",split_attribute_name)
    print("target", target_attribute_name)
    print("novs",nobs)
    print("df_split",df_split)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    print(df_agg_ent)
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


In [6]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    if len(cnt) == 1:
        return next(iter(cnt))  # next input data set, or raises StopIteration when EOF is hit.
    elif df.empty or (not attribute_names):
        return default_class  # Return None for Empty Data Set
    else:
        default_class = max(cnt.keys()) #No of YES and NO Class
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute_name, remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree
        return tree


In [7]:
attribute_names = list(df_tennis.columns)
attribute_names.remove('Y') #Remove the class attribute 
# Run Algorithm:
from pprint import pprint
tree = id3(df_tennis,'Y',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)


split A
target Y
novs 4.0
df_split <pandas.core.groupby.generic.DataFrameGroupBy object at 0x049567A8>
   Entropy  PropObservations
A                           
0        0               0.5
1        1               0.5
split B
target Y
novs 4.0
df_split <pandas.core.groupby.generic.DataFrameGroupBy object at 0x111CBF10>
   Entropy  PropObservations
B                           
0        1               0.5
1        0               0.5
split B
target Y
novs 2.0
df_split <pandas.core.groupby.generic.DataFrameGroupBy object at 0x04956448>
   Entropy  PropObservations
B                           
0        0               0.5
1        0               0.5


The Resultant Decision Tree is :

{'A': {0: 0, 1: {'B': {0: 1, 1: 0}}}}
